# Расчет результатов

1. Посчитать модели линейной регрессии для 20 зданий по оптимальному набору параметров: метеорологические данные, дни недели, недели года, месяцы и праздники по всему набору данных.
2. Загрузить данные решения, посчитать значение энергопотребления для требуемых дат для тех зданий, которые посчитаны в модели, и выгрузить результат в файл.

## Подключение библиотек

In [1]:
import pandas as pd
import numpy as np

from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from scipy.interpolate import interp1d
from sklearn.linear_model import LinearRegression

from core.reduce_mem_usage import reduce_mem_usage

## Загрузка данных 20 зданий из HDF5

In [2]:
energy = pd.read_hdf("../data/out/energy.0-20.h5", "energy")
print(energy.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 175680 entries, 0 to 175679
Data columns (total 92 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   timestamp               175680 non-null  datetime64[ns]
 1   building_id             175680 non-null  int8          
 2   meter_reading           175680 non-null  float16       
 3   primary_use             175680 non-null  category      
 4   air_temperature         175680 non-null  float16       
 5   cloud_coverage          175680 non-null  float16       
 6   dew_temperature         175680 non-null  float16       
 7   precip_depth_1_hr       175680 non-null  float16       
 8   sea_level_pressure      175680 non-null  float16       
 9   wind_direction          175680 non-null  float16       
 10  wind_speed              175680 non-null  float16       
 11  air_temperature_diff_1  175680 non-null  float16       
 12  air_temperature_diff_2  175680

## Загрузка данных и оптимизация памяти

In [3]:
buidings = pd.read_csv("../data/buildings.csv", usecols=["site_id", "building_id"])

weather = pd.read_csv("../data/weather_test.csv.gz")
weather = weather[weather["site_id"] == 0]
weather = weather.drop(columns=["wind_direction"], axis=1)

results = pd.read_csv("../data/test.parquet.gzip")
results = results[(results["building_id"] < 20) & (results["meter"] == 0)]
results = pd.merge(
    left=results,
    right=buidings,
    how="left",
    left_on="building_id",
    right_on="building_id",
)

del buidings

results = results.drop(columns=["meter"], axis=1)
print(results.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 350400 entries, 0 to 350399
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   row_id       350400 non-null  int64 
 1   building_id  350400 non-null  int64 
 2   timestamp    350400 non-null  object
 3   site_id      350400 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 13.4+ MB
None


## Интерполяция значений и обогащение погодных данных для одного года

In [5]:
interpolate_columns = [
    "air_temperature",
    "dew_temperature",
    "cloud_coverage",
    "wind_speed",
    "sea_level_pressure",
]

for col in interpolate_columns:
    weather[col] = weather[col].interpolate(limit_direction="both", kind="cubic")

weather["air_temperature_diff_1"] = weather["air_temperature"].diff()
weather.at[0, "air_temperature_diff_1"] = weather.at[1, "air_temperature_diff_1"]
weather["air_temperature_diff_2"] = weather["air_temperature_diff_1"].diff()
weather.at[0, "air_temperature_diff_2"] = weather.at[1, "air_temperature_diff_2"]

## Объединение данных по погоде

In [6]:
results = results.set_index(["timestamp", "site_id"])
weather = weather.set_index(["timestamp", "site_id"])

results = pd.merge(
    left=results,
    right=weather,
    how="left",
    left_index=True,
    right_index=True,
)

results.reset_index(inplace=True)
results = results.drop(columns=["site_id"], axis=1)

del weather

results = reduce_mem_usage(results)
print(results.info())

Потребление памяти меньше на - 19.72 Мб (минус 67.0%)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350400 entries, 0 to 350399
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   timestamp               350400 non-null  datetime64[ns]
 1   row_id                  350400 non-null  int32         
 2   building_id             350400 non-null  int8          
 3   air_temperature         350400 non-null  float16       
 4   cloud_coverage          350400 non-null  float16       
 5   dew_temperature         350400 non-null  float16       
 6   precip_depth_1_hr       349800 non-null  float16       
 7   sea_level_pressure      350400 non-null  float16       
 8   wind_speed              350400 non-null  float16       
 9   air_temperature_diff_1  350400 non-null  float16       
 10  air_temperature_diff_2  350400 non-null  float16       
dtypes: datetime64[ns](1), float16(8), int

## Обогащение данных по дате

In [7]:
results["hour"] = results["timestamp"].dt.hour.astype("int8")
results["weekday"] =results["timestamp"].dt.weekday.astype("int8")
results["week"] =results["timestamp"].dt.isocalendar().week.astype("int8")
results["month"] =results["timestamp"].dt.month.astype("int8")
results["date"] = pd.to_datetime(results["timestamp"].dt.date)

dates_range = pd.date_range(start="2016-12-31", end="2018-06-01")
us_holidays = calendar().holidays(start=dates_range.min(), end=dates_range.max())
results["is_holiday"] = results["date"].isin(us_holidays).astype("int8")

for weekday in range(0, 7):
    results[f"is_wday {str(weekday)}"] = results["weekday"].isin([weekday]).astype("int8")
for week in range(1, 54):
    results[f"is_w {str(week)}"] = results["week"].isin([week]).astype("int8")
for month in range(1, 13):
    results[f"is_m {str(month)}"] = results["month"].isin([month]).astype("int8")

## Линейная регрессия

In [8]:
hours = range(0, 24)
buildings = range(0, energy["building_id"].max() + 1)
lr_columns = [
    "meter_reading_log",
    "hour",
    "building_id",
    "air_temperature",
    "dew_temperature",
    "sea_level_pressure",
    "wind_speed",
    "cloud_coverage",
    "air_temperature_diff_1",
    "air_temperature_diff_2",
    "is_holiday",
]

for wday in range(0, 7):
    lr_columns.append(f"is_wday {str(wday)}")
for week in range(1, 54):
    lr_columns.append(f"is_w {str(week)}")
for month in range(1, 13):
    lr_columns.append(f"is_m {str(month)}")

energy_train_lr = pd.DataFrame(energy, columns=lr_columns)
energy_lr = [[] for _ in range(len(buildings))]

for building in buildings:
    energy_lr[building] = [[] for _ in range(len(hours))]
    energy_train_b = energy_train_lr[energy_train_lr["building_id"] == building]

    for hour in hours:
        energy_train_bh = energy_train_b[energy_train_b["hour"] == hour]
        y = energy_train_bh["meter_reading_log"]
        x = energy_train_bh.drop(
            labels=["meter_reading_log",
                    "hour",
                    "building_id"],
            axis=1
        )
        model = LinearRegression(fit_intercept=False).fit(x, y)
        energy_lr[building][hour] = model.coef_
        energy_lr[building][hour] = np.append(
            energy_lr[building][hour],
            model.intercept_
        )

print(energy_lr[0])

[array([ 1.82714749e-02, -5.00985631e-03, -5.45449149e-03, -2.27989402e-02,
       -1.66069846e-02,  2.12089215e-03,  7.38366143e-03,  1.32797603e-02,
        5.04346179e+00,  5.06167951e+00,  5.08275128e+00,  5.04801097e+00,
        5.05671254e+00,  5.15887071e+00,  5.14694236e+00,  1.12637660e-01,
        6.23951661e-02,  8.97421800e-02,  9.06670422e-02, -1.86559587e+00,
       -1.85568158e+00, -1.87255580e+00, -1.86312791e+00, -1.80246165e+00,
       -1.76412676e+00, -1.83587174e+00, -1.73135377e+00, -1.71595247e+00,
       -1.64919738e+00, -1.55253680e+00, -1.58911640e+00, -1.58752247e+00,
       -1.55544515e+00, -1.53436302e+00,  4.05354240e-02,  3.74750803e+00,
        3.67292119e+00,  3.80375247e+00,  3.84794737e+00,  3.71133383e+00,
        3.75950840e+00,  3.67976173e+00,  3.87701370e+00,  3.88738653e+00,
        3.81739851e+00,  1.05369630e+00,  1.16506097e+00,  1.15678713e+00,
        9.92240842e-01,  9.99323814e-01,  9.91959249e-01,  9.67509387e-01,
        9.37870456e-01, 

## Расчет финальных показателей, только энергопотребление 20 первых зданий

In [9]:
def calculate_model(x):
    lr = -1
    model = energy_lr[x.building_id][x.hour]
    if len(model) > 0:
        lr = np.sum([x[col] * model[i] for i, col in enumerate(lr_columns[3:])])
        lr +=model[len(lr_columns) - 3]
        lr = np.exp(lr)
    if lr < 0 or lr != lr or lr*lr == lr:
        lr =0
    x["meter_reading"] = lr
    if x["row_id"] % 1000 == 0:
        print(f"Готово {x['row_id']}")
    return x

In [10]:
results = results.apply(calculate_model, axis=1, result_type="expand")

Готово 0
Готово 4000
Готово 8000
Готово 12000
Готово 16000
Готово 20000
Готово 24000
Готово 28000
Готово 36000
Готово 40000
Готово 48000
Готово 52000
Готово 56000
Готово 60000
Готово 76000
Готово 84000
Готово 88000
Готово 92000
Готово 96000
Готово 129000
Готово 133000
Готово 137000
Готово 141000
Готово 145000
Готово 149000
Готово 153000
Готово 157000
Готово 165000
Готово 169000
Готово 177000
Готово 181000
Готово 185000
Готово 189000
Готово 197000
Готово 205000
Готово 213000
Готово 217000
Готово 221000
Готово 225000
Готово 258000
Готово 262000
Готово 266000
Готово 270000
Готово 274000
Готово 278000
Готово 282000
Готово 286000
Готово 294000
Готово 298000
Готово 306000
Готово 310000
Готово 314000
Готово 318000
Готово 326000
Готово 334000
Готово 342000
Готово 346000
Готово 350000
Готово 354000
Готово 387000
Готово 391000
Готово 395000
Готово 399000
Готово 403000
Готово 407000
Готово 411000
Готово 415000
Готово 423000
Готово 427000
Готово 435000
Готово 439000
Готово 443000
Готово 447000
Гот

## Усечение данных до требуемого формата

In [11]:
results_ready = pd.DataFrame(results, columns=["row_id", "meter_reading"])

## Загрузка всех данных для заполнения их нулями

In [13]:
results = pd.read_csv("../data/test.parquet.gzip", usecols=["row_id"])
results = pd.merge(
    left=results,
    right=results_ready,
    how="left",
    left_on="row_id",
    right_on="row_id",
)
results.fillna(value=0, inplace=True)
print(results.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41697600 entries, 0 to 41697599
Data columns (total 2 columns):
 #   Column         Dtype  
---  ------         -----  
 0   row_id         int64  
 1   meter_reading  float64
dtypes: float64(1), int64(1)
memory usage: 954.4 MB
None


## Выгрузка результатов в CSV файл

In [15]:
results.to_csv("../data/out/submission.csv.gz", index=False, compression="gzip")

## Освободить память

In [16]:
del energy
del results
del results_ready